In [3]:
import scipy.io as io
# read data/zuco/task1/Matlab_files/resultsZAB_SR.mat
sentence_data = io.loadmat('data/zuco/task1/Matlab_files/resultsZAB_SR.mat', squeeze_me=True, struct_as_record=False,)['sentenceData']

In [6]:
import numpy as np
import pandas as pd
sn_data = sentence_data[0]
dur = np.array(sn_data.allFixations.duration)
x = np.array(sn_data.allFixations.x)
y = np.array(sn_data.allFixations.y)


In [12]:
sn_data.content

'Presents a good case while failing to provide a reason for us to care beyond the very basic dictums of human decency.'

In [ ]:
word_boundary = sn_data.wordbounds
df = pd.DataFrame([], columns=['sn', 'wn', 'fl', 'dur'])
for fix_idx in range(x.shape[0]):
    word_idx = np.where(
        (word_boundary[:, 0] <= x[fix_idx]) &
        (word_boundary[:, 2] >= x[fix_idx]) &
        (word_boundary[:, 1] <= y[fix_idx]) &
        (word_boundary[:, 3] >= y[fix_idx]),
    )
    assert len(word_idx) == 1, 'more than 1 word is matched!'
    # skip fixations outside all boundary boxes
    if word_idx[0].size == 0:
        continue
    fl = ((x[fix_idx] - word_boundary[word_idx, 0]) /
            (word_boundary[word_idx, 2] - word_boundary[word_idx, 0]))[0][0]
    df_tmp = pd.DataFrame(
        [[
            1,  word_idx[0][0] + 1,
            fl, dur[fix_idx],
        ]], columns=['sn', 'wn', 'fl', 'dur'],
    )
    df = pd.concat([df, df_tmp])


In [11]:
words = pd.read_csv("data/zuco/task1/Matlab_files/scanpath.csv", sep='\t')
# check is there is nan in words
words.isnull().values.any()

# check if there is nan or '' in column 'WORD'
words[words['CURRENT_FIX_INTEREST_AREA_LABEL'].isnull().values]
words[words['CURRENT_FIX_INTEREST_AREA_LABEL'] == '']

,id,sn,nw,wn,fl,dur,CURRENT_FIX_INTEREST_AREA_LABEL


In [5]:
import pandas as pd
path = "data/zuco/task2/Matlab_files/scanpath.csv"
df = pd.read_csv(path, sep='\t')
df['id'].unique()


array(['ZAB', 'ZDM', 'ZDN', 'ZGW', 'ZJM', 'ZJN', 'ZJS', 'ZKB', 'ZKH',
       'ZKW', 'ZMG', 'ZPH'], dtype=object)

In [30]:
import pandas as pd
import numpy as np
import ast

df = pd.read_csv("data/zuco/task2/fixation.csv", sep=',')
sentences = df['sn'].unique()
df_avg = pd.DataFrame([], columns=['sn', 'list_dur'])

for sn in sentences:
    df_tmp = df[df['sn'] == sn]
    # Convert string representations to actual lists
    df_tmp['list_dur'] = df_tmp['list_dur'].apply(ast.literal_eval)
    
    # Calculate the average fixation duration
    avg_dur = np.nanmean(df_tmp['list_dur'].values.tolist(), axis=0)

    df_avg = pd.concat([df_avg, pd.DataFrame({'sn': [sn], 'list_dur': [avg_dur.tolist()]})], ignore_index=True)

df_avg.to_csv("data/zuco/task2/fixation_avg.csv", index=False)

    


/var/folders/ng/jh0twkn55sbds4qg4kkndgq80000gn/T/ipykernel_6601/2755516044.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tmp['list_dur'] = df_tmp['list_dur'].apply(ast.literal_eval)
/var/folders/ng/jh0twkn55sbds4qg4kkndgq80000gn/T/ipykernel_6601/2755516044.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tmp['list_dur'] = df_tmp['list_dur'].apply(ast.literal_eval)
/var/folders/ng/jh0twkn55sbds4qg4kkndgq80000gn/T/ipykernel_6601/2755516044.py:12: SettingWithCopyWarning: 
A value is trying to 

In [ ]:
a = [1, 2, 3]
b = [4, 5, 6]

df = pd.DataFrame(columns=['list'])
df['list'] = [a]
df['list'].append({'list': b})
df



In [22]:
from transformers import BertTokenizer, BertForMaskedLM
import torch
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
text = "Ford was born on a prosperous farm in Springwells Township (now in the city of Dearborn, Michigan) owned by his parents, William Ford (1826-1905) and Mary Litogot (c1839-1876), immigrants from County Cork, Ireland." 
text = text.strip() + "[MASK]" * 10
input_tokens = tokenizer(text)['input_ids']
attention_ids = tokenizer(text)['attention_mask']
input_tokens = torch.tensor(input_tokens).unsqueeze(0)
attention_ids = torch.tensor(attention_ids).unsqueeze(0)
torch.enable_grad()
model.eval()

A = model(input_tokens, attention_mask=attention_ids)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [39]:
import numpy as np
import string

def merge_bert_tokens(tokens, text, gradients, special_tokens=["[CLS]", "[SEP]", "[MASK]"]):
    gradients_list = []
    word_list = []
    word = ""
    word_gradients = 0
    # Merge tokens into the original words
    for i, token in enumerate(tokens):
        if token in special_tokens:
            continue
        if token.startswith("##"):
            word += token[2:]
            word_gradients += gradients[i]
        else:
            if word != "":
                word_list.append(word)
                gradients_list.append(word_gradients)
                word = ""
                word_gradients = 0
            word = token
            word_gradients = gradients[i]
    if word != "":
        word_list.append(word)
        gradients_list.append(word_gradients)
    # Merge further the words into the original text

    merged_gradients, merged_word_list = merge_further(word_list, text, gradients_list)

    return np.array(merged_gradients).squeeze(), merged_word_list


               

# Example usage:
bert_tokens = tokenizer.convert_ids_to_tokens(input_tokens[0])
# generate a randon gradient list with the same length of bert_tokens
random_gradients = np.random.rand(len(bert_tokens))
print(bert_tokens)

merged_gradients_bert, word_list = merge_bert_tokens(bert_tokens, text, random_gradients)
print(merged_gradients_bert)
print(word_list)


['[CLS]', 'ford', 'was', 'born', 'on', 'a', 'prosperous', 'farm', 'in', 'spring', '##well', '##s', 'township', '(', 'now', 'in', 'the', 'city', 'of', 'dear', '##born', ',', 'michigan', ')', 'owned', 'by', 'his', 'parents', ',', 'william', 'ford', '(', '1826', '-', '1905', ')', 'and', 'mary', 'lit', '##ogo', '##t', '(', 'c1', '##8', '##39', '-', '1876', ')', ',', 'immigrants', 'from', 'county', 'cork', ',', 'ireland', '.', '[MASK]', '[MASK]', '[MASK]', '[MASK]', '[MASK]', '[MASK]', '[MASK]', '[MASK]', '[MASK]', '[MASK]', '[SEP]']
['ford', 'was', 'born', 'on', 'a', 'prosperous', 'farm', 'in', 'springwells', 'township', '(now', 'in', 'the', 'city', 'of', 'dearborn,', 'michigan)', 'owned', 'by', 'his', 'parents,', 'william', 'ford', '(1826-1905)', 'and', 'mary', 'litogot', '(c1839-1876),', 'immigrants', 'from', 'county', 'cork,', 'ireland.']
target:  ford
word:  ford
target:  was
word:  was
target:  born
word:  born
target:  on
word:  on
target:  a
word:  a
target:  prosperous
word:  prosp

In [35]:
def merge_further(word_list, text, gradients):
    original_wrods = text.lower().split()
    print(original_wrods)
    merged_word_list = []
    merged_gradients = []
    tmp = ""
    word_gradients = 0
    word_count = 0
    
    for target in original_wrods:     
        
        for i, word in enumerate(word_list[word_count:]):
            
            if word == target:
                merged_word_list.append(word)
                merged_gradients.append(gradients[i])
                word_count += 1
                break
            elif word in target:
                tmp += word
                word_gradients += gradients[i]
                word_count += 1
            else:
                merged_word_list.append(tmp)
                merged_gradients.append(word_gradients)
                tmp = ""
                word_gradients = 0
                break
    if tmp != "":
        merged_word_list.append(tmp)
        merged_gradients.append(word_gradients)
                
    return merged_gradients, merged_word_list

text = "Ford was born on a prosperous farm in Springwells Township (now in the city of Dearborn, Michigan) owned by his parents, William Ford (1826-1905) and Mary Litogot (c1839-1876), immigrants from County Cork, Ireland."
merged_gradients, merged_word_list = merge_further(word_list, text, merged_gradients_bert)
print(merged_gradients)
print(merged_word_list)
print(len(merged_gradients))

['ford', 'was', 'born', 'on', 'a', 'prosperous', 'farm', 'in', 'springwells', 'township', '(now', 'in', 'the', 'city', 'of', 'dearborn,', 'michigan)', 'owned', 'by', 'his', 'parents,', 'william', 'ford', '(1826-1905)', 'and', 'mary', 'litogot', '(c1839-1876),', 'immigrants', 'from', 'county', 'cork,', 'ireland.']
target:  ford
word:  ford
target:  was
word:  was
target:  born
word:  born
target:  on
word:  on
target:  a
word:  a
target:  prosperous
word:  prosperous
target:  farm
word:  farm
target:  in
word:  in
target:  springwells
word:  springwells
target:  township
word:  township
target:  (now
word:  (
word:  now
word:  in
target:  in
word:  in
target:  the
word:  the
target:  city
word:  city
target:  of
word:  of
target:  dearborn,
word:  dearborn
word:  ,
word:  michigan
target:  michigan)
word:  michigan
word:  )
word:  owned
target:  owned
word:  owned
target:  by
word:  by
target:  his
word:  his
target:  parents,
word:  parents
word:  ,
word:  william
target:  william
word